# Lire les données et les afficher (repris du code de Olivier)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split  

path='/Users/marierebiere/Desktop/cours/supaero/4A/Bigdata2/public_dat/'  # si les données sont déjà dans le répertoire courant

input_string_train = path + 'store_train.data'
#df_train = pd.DataFrame([x.split(' ') for x in input_string_train.split('\n')],columns=["ID","Day","Date","Number_of_Customers","Open","Promo","State_Holiday","School_Holiday"],
                  #dtype=int)
                  #{"ID":int,"Day":int,"Date":object,"Number_of_Customers":int,"Open":int,"Promo":int,"State_Holiday":int,"School_Holiday":int})
df_train = pd.read_csv(input_string_train ,sep=" ",header=None, names = ["ID", "Day", "Date", "Number_of_Customers", "Open", "Promo", "State_Holiday", "School_Holiday"])

df_train["Date"]=pd.DataFrame(df_train["Date"],dtype="datetime64[ns]")
df_train["State_Holiday"]=pd.DataFrame(df_train["State_Holiday"],dtype="bool")
df_train["State_Holiday"]=pd.DataFrame(df_train["State_Holiday"],dtype="int")

df_train['Month'] = df_train['Date'].map(lambda d: d.month)
df_train["Month"]=pd.DataFrame(df_train["Month"],dtype=int)
df_train['Day_of_month'] = df_train['Date'].map(lambda d: d.day)
df_train["Day_of_month"]=pd.DataFrame(df_train["Day_of_month"],dtype=int)
df_train['Year'] = df_train['Date'].map(lambda d: d.year)
df_train["Year"]=pd.DataFrame(df_train["Year"],dtype=int)

del df_train["Date"]

# Lire les données de test
input_string_test = path + 'store_test.data'
df_test = pd.read_csv(input_string_test ,sep=" ",header=None, names = ["ID", "Day", "Date", "Number_of_Customers", "Open", "Promo", "State_Holiday", "School_Holiday"])
df_test["Date"]=pd.DataFrame(df_test["Date"],dtype="datetime64[ns]")
df_test["State_Holiday"]=pd.DataFrame(df_test["State_Holiday"],dtype="bool")
df_test["State_Holiday"]=pd.DataFrame(df_test["State_Holiday"],dtype="int")

df_test['Month'] = df_test['Date'].map(lambda d: d.month)
df_test["Month"]=pd.DataFrame(df_test["Month"],dtype=int)
df_test['Day_of_month'] = df_test['Date'].map(lambda d: d.day)
df_test["Day_of_month"]=pd.DataFrame(df_test["Day_of_month"],dtype=int)
df_test['Year'] = df_test['Date'].map(lambda d: d.year)
df_test["Year"]=pd.DataFrame(df_test["Year"],dtype=int)

del df_test["Date"]

##### solutions
input_string_sol = path + 'store_train.solution'
df_sol = pd.read_csv(input_string_sol ,sep=" ",header=None, names = ["Sales"])


df_full_train=pd.concat([df_train,df_sol],axis=1)

print("Données d'entrainement complètes :\n", df_full_train.head(), "\n")
print("Données de test :\n", df_test.head(), "\n")
print("Types des données d'entrainement complètes : \n", df_full_train.dtypes)

Données d'entrainement complètes :
     ID  Day  Number_of_Customers  Open  Promo  State_Holiday  School_Holiday  \
0  625    3                  641     1      1              1               0   
1  293    2                  877     1      1              1               1   
2   39    4                  561     1      1              1               0   
3  676    4                 1584     1      1              1               0   
4  709    3                 1477     1      1              1               0   

   Month  Day_of_month  Year  Sales  
0     11             6  2013   7293  
1      7            16  2013   7060  
2      1            23  2014   4565  
3      9            26  2013   6380  
4      1            22  2014  11647   

Données de test :
     ID  Day  Number_of_Customers  Open  Promo  State_Holiday  School_Holiday  \
0  249    5                  725     1      1              1               0   
1  190    4                  564     1      0              1              

On vérifie que le tableau est bien rempli

In [2]:
print("Nb de jours", df_full_train["Day"].size)
print("Nb de magasins", df_full_train["ID"].size)
print("Nb de Customers", df_full_train["Number_of_Customers"].size)
print("Nb de open", df_full_train["Open"].size)
print("Nb de Promo", df_full_train["Promo"].size)
print("Nb de State_Holiday", df_full_train["State_Holiday"].size)
print("Nb de School_Holiday", df_full_train["School_Holiday"].size)
print("Nb de Sales", df_full_train["Sales"].size)

Day_nul = df_full_train.isna()["Day"]
print(df_full_train.isna())
print(Day_nul[1])
nb_jour_nn_remplis = 0
for i in range(df_full_train["Day"].size-1) :
    if Day_nul[i] :
        nb_jour_nn_remplis +1

print("le nombre de jours non remplis est de ",nb_jour_nn_remplis)


Nb de jours 712045
Nb de magasins 712045
Nb de Customers 712045
Nb de open 712045
Nb de Promo 712045
Nb de State_Holiday 712045
Nb de School_Holiday 712045
Nb de Sales 712045
           ID    Day  Number_of_Customers   Open  Promo  State_Holiday  \
0       False  False                False  False  False          False   
1       False  False                False  False  False          False   
2       False  False                False  False  False          False   
3       False  False                False  False  False          False   
4       False  False                False  False  False          False   
5       False  False                False  False  False          False   
6       False  False                False  False  False          False   
7       False  False                False  False  False          False   
8       False  False                False  False  False          False   
9       False  False                False  False  False          False   
10      Fal

le nombre de jours non remplis est de  0


Découper les données de test et d'apprentissage pour voir l'intéret des différents algorithmes sur notre propre set d'entrainement

In [3]:
from sklearn.model_selection import train_test_split
T_train,T_test,z_train,z_test=train_test_split(df_train,df_sol,test_size=0.2,random_state=11)

## Algo KNN pour identifier les jours où il y a vente et les jours où on ne vend rien

Pour celà il faut redéfinir les données de la solution en variable binaire : 0 si pas de vente et 1 si la vente>0.

In [9]:
print(df_sol[:10])
df_sol2=pd.DataFrame(df_sol,dtype="bool")
df_sol2=pd.DataFrame(df_sol2,dtype="int")
print(df_sol2[:10])

from sklearn.model_selection import train_test_split
X_train,X_test,z_train,z_test=train_test_split(df_train,df_sol2,test_size=0.25)

   Sales
0   7293
1   7060
2   4565
3   6380
4  11647
5   7967
6   7101
7      0
8   7516
9   6295
   Sales
0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      1


In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
digit_knn=knn.fit(X_train, z_train) 
# Estimation de l'erreur de prévision
# sur l'échantillon test
1-digit_knn.score(X_test,z_test)


/Users/marierebiere/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


5.617598813567604e-06

Le score obtenu n'est pas ouf et ne change pas en fonction du nombre de voisins c'est chelou

In [11]:
from sklearn.model_selection import GridSearchCV
# grille de valeurs
param=[{"n_neighbors":list(range(1,15))}]
knn= GridSearchCV(KNeighborsClassifier(),param,cv=5,n_jobs=-1)
digit_knnOpt=knn.fit(X_train, z_train)
# paramètre optimal
digit_knnOpt.best_params_["n_neighbors"]

/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


1

In [14]:
z_test_tolist = z_test.values.tolist()

z_pred = digit_knnOpt.predict(X_test)

a = np.array(z_test_tolist) # your x
b = np.array(z_pred) # your y
mse = 0
for i in range(len(a)):
    mse+= (a[i]-b[i])**2/len(a)

mse

array([5.61759881e-06])

## Algo KNN pour les plages de vente : 

In [25]:
print(df_sol[:10])
df_sol1=df_sol
df_sol1["Sales"]=pd.qcut(df_sol["Sales"],3,labels=["Sales1","Sales2","Sales3"])
print(df_sol1[:10])

#from sklearn.model_selection import train_test_split
#X_train,X_test,z_train,z_test=train_test_split(df_train,df_sol2,test_size=0.25)

   Sales
0   7293
1   7060
2   4565
3   6380
4  11647
5   7967
6   7101
7      0
8   7516
9   6295
    Sales
0  Sales3
1  Sales3
2  Sales2
3  Sales2
4  Sales3
5  Sales3
6  Sales3
7  Sales1
8  Sales3
9  Sales2


## Régression logistique

In [16]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
Sales_logit=logit.fit(X_train, z_train)
Sales_logit.predict(X_test)
# Erreur sur l'écahntillon test
#1-titan_logit.score(T_test, z_test)

/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/marierebiere/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 1, ..., 1, 0, 1])